In [1]:
!pip install pyspark
!pip install findspark

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

In [4]:
# 𝐐𝐮𝐞𝐬𝐭𝐢𝐨𝐧
# You are working as a Data Engineer on a big data processing pipeline, 
# where you need to analyze a large dataset from a retail business. 
# This dataset contains transaction information of customers, including a unique transaction ID, customer ID, product ID, transaction amount, and date. 
# The dataset has an index column, but you are required to include this index as a regular column in the DataFrame to perform some further transformations and joins. 
# Your task is to convert the index of the PySpark DataFrame into a column while preserving the dataset's integrity.

In [5]:
spark = SparkSession.builder.appName('example').getOrCreate()
data = [ (1001, 5001, 200, "2025-01-01"), 
(1002, 5002, 450, "2025-01-02"), 
(1003, 5003, 300, "2025-01-03"), 
(1004, 5004, 150, "2025-01-04"), 
(1005, 5005, 500, "2025-01-05"), ] 

# Schema for DataFrame 
columns = ["Customer_ID", "Product_ID", "Amount", "Date"] 

df = spark.createDataFrame(data=data, schema=columns)
df.show()

25/04/26 18:18:54 WARN Utils: Your hostname, MC1275 resolves to a loopback address: 127.0.1.1; using 192.168.29.179 instead (on interface wlp0s20f3)
25/04/26 18:18:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/26 18:18:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/26 18:18:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/04/26 18:18:55 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/04/26 18:18:55 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/04/26 18:18:55 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


+-----------+----------+------+----------+
|Customer_ID|Product_ID|Amount|      Date|
+-----------+----------+------+----------+
|       1001|      5001|   200|2025-01-01|
|       1002|      5002|   450|2025-01-02|
|       1003|      5003|   300|2025-01-03|
|       1004|      5004|   150|2025-01-04|
|       1005|      5005|   500|2025-01-05|
+-----------+----------+------+----------+



In [6]:
# Define a window (no partition, order by Date or any column that ensures order)
windowSpec = Window.orderBy("Date")

25/04/26 18:19:10 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [7]:
# Add index column
df_with_index = df.withColumn("Index_Column", row_number().over(windowSpec) - 1)

In [8]:
df_with_index.show(truncate=False)

25/04/26 18:19:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/26 18:19:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/26 18:19:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+----------+------+----------+------------+
|Customer_ID|Product_ID|Amount|Date      |Index_Column|
+-----------+----------+------+----------+------------+
|1001       |5001      |200   |2025-01-01|0           |
|1002       |5002      |450   |2025-01-02|1           |
|1003       |5003      |300   |2025-01-03|2           |
|1004       |5004      |150   |2025-01-04|3           |
|1005       |5005      |500   |2025-01-05|4           |
+-----------+----------+------+----------+------------+



In [12]:
# Convert to SQL 
df.createOrReplaceTempView("customers")

In [11]:
spark.sql("SELECT * FROM customers").show()

+-----------+----------+------+----------+
|Customer_ID|Product_ID|Amount|      Date|
+-----------+----------+------+----------+
|       1001|      5001|   200|2025-01-01|
|       1002|      5002|   450|2025-01-02|
|       1003|      5003|   300|2025-01-03|
|       1004|      5004|   150|2025-01-04|
|       1005|      5005|   500|2025-01-05|
+-----------+----------+------+----------+



In [15]:
spark.sql("""
            SELECT 
            *, ROW_NUMBER() OVER (ORDER by DATE) - 1 as Index_column
            
            FROM customers
        """).show()

25/04/26 19:02:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/26 19:02:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/26 19:02:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+----------+------+----------+------------+
|Customer_ID|Product_ID|Amount|      Date|Index_column|
+-----------+----------+------+----------+------------+
|       1001|      5001|   200|2025-01-01|           0|
|       1002|      5002|   450|2025-01-02|           1|
|       1003|      5003|   300|2025-01-03|           2|
|       1004|      5004|   150|2025-01-04|           3|
|       1005|      5005|   500|2025-01-05|           4|
+-----------+----------+------+----------+------------+



In [ ]:
s